In [11]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score,precision_recall_fscore_support
from Constants import *
from DeepSurvivalModels import *
from Utils import *
from Preprocessing import DTDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
data = DTDataset(use_smote=False)
data.processed_df[Const.timeseries_outcomes]

,OS (Calculated),Locoregional control (Time),FDM (months),time_to_event
id,,,,
3,6.033333,4.700000,6.033333,4.700000
5,7.333333,7.333333,7.333333,6.000000
6,7.466667,7.466667,7.466667,6.000000
7,7.800000,7.800000,7.800000,6.000000
8,8.066667,8.066667,8.066667,8.066667
...,...,...,...,...
10201,143.200000,143.200000,143.200000,143.200000
10202,144.366667,144.366667,144.366667,6.000000
10203,148.366667,148.366667,136.033333,136.033333


In [13]:
xtr,xtst,ytr,ytst = transition_sample(3,data)
xtr.shape

torch.Size([389, 83])

In [14]:
test = DSM(xtr.shape[1],dist="LogNormal")
test.fit_normalizer(xtr)
test.time_to_event(xtr)

[tensor([1.1004, 0.8636, 0.9715, 1.1178, 0.8600, 1.3957, 0.8527, 1.4517, 1.1329,
         0.8933, 1.5627, 1.1706, 1.6400, 1.3307, 1.2273, 1.1290, 1.0910, 0.9068,
         1.0170, 1.3414, 0.8026, 1.3091, 1.4475, 1.9470, 1.3535, 1.0268, 1.4125,
         1.2469, 1.0580, 1.0986, 0.8409, 1.0500, 1.0022, 1.1186, 1.2320, 1.0576,
         1.1122, 0.8575, 1.2904, 1.1238, 0.8699, 1.1697, 0.9525, 1.1163, 0.9444,
         1.2880, 1.4202, 1.0301, 1.0856, 0.9662, 1.2181, 1.0702, 1.1124, 1.8604,
         0.9908, 1.0566, 1.5701, 1.4657, 1.2799, 1.0452, 0.9660, 1.0539, 1.0828,
         1.1101, 1.1370, 1.1889, 1.2088, 1.0423, 1.0818, 1.0525, 1.1530, 1.2939,
         1.3948, 1.3909, 1.1185, 1.0728, 1.2159, 1.1636, 1.2204, 1.0579, 1.3843,
         1.2640, 1.6423, 1.1228, 1.0838, 1.1013, 1.4181, 1.4676, 1.1940, 0.9697,
         1.2908, 1.0870, 1.8451, 1.2115, 1.1880, 1.1583, 1.1975, 1.0191, 0.9776,
         1.0709, 1.1593, 1.0671, 1.1669, 1.0539, 1.2470, 1.1230, 0.8821, 1.6663,
         0.9843, 1.2374, 1.2

In [15]:
test = train_dsm(data,k=10,dist="Weibull")
test.time_to_event(xtr)

events torch.Size([4, 389])
events torch.Size([4, 147])
events torch.Size([4, 389])
best pretrain loss 46012.893248336935 epochs 1315
val loss 2.8362384971720904
val metrics {'OS (Calculated)': {12: {'roc_score': 0.9265734265734265, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9766042780748663, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.9833524684270952, 'f1': 0.9571428571428571, 'matthews': 0.26570779485078694}, 48: {'roc_score': 0.9608778625954199, 'f1': 0.962406015037594, 'matthews': 0.6137753724313343}}, 'Locoregional control (Time)': {12: {'roc_score': 0.8839928057553956, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.8821564885496183, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.8988188976377952, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.9025245441795231, 'f1': 0.9253731343283582, 'matthews': 0.33514003129991815}}, 'FDM (months)': {12: {'roc_score': 0.9125295508274232, 'f1': 0.9791666

/data/DigitalTwinVis/python/DeepSurvivalModels.py:378: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  survival = [torch.exp(c).T for c in cdf]


val loss 2.6505638873689694
val metrics {'OS (Calculated)': {12: {'roc_score': 0.9423076923076923, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9712566844919786, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.9781859931113662, 'f1': 0.9571428571428571, 'matthews': 0.26570779485078694}, 48: {'roc_score': 0.9666030534351144, 'f1': 0.9594095940959411, 'matthews': 0.5372569573006374}}, 'Locoregional control (Time)': {12: {'roc_score': 0.9199640287769785, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.9174618320610687, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.9149606299212598, 'f1': 0.9473684210526315, 'matthews': 0.5170714090044591}, 48: {'roc_score': 0.9211079943899018, 'f1': 0.9523809523809523, 'matthews': 0.6713140161686046}}, 'FDM (months)': {12: {'roc_score': 0.9267139479905437, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.9644087256027554, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'r

val loss 2.330283270792874
val metrics {'OS (Calculated)': {12: {'roc_score': 0.9353146853146853, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9197860962566845, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.936280137772675, 'f1': 0.9571428571428571, 'matthews': 0.26570779485078694}, 48: {'roc_score': 0.9289122137404581, 'f1': 0.962121212121212, 'matthews': 0.630714136050733}}, 'Locoregional control (Time)': {12: {'roc_score': 0.9658273381294964, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.9413167938931297, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.9409448818897639, 'f1': 0.9407407407407408, 'matthews': 0.4214526991518474}, 48: {'roc_score': 0.9442496493688639, 'f1': 0.8761061946902654, 'matthews': 0.5265187925285103}}, 'FDM (months)': {12: {'roc_score': 0.897163120567376, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.9230769230769231, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_sc

val loss 2.3779229618190567
val metrics {'OS (Calculated)': {12: {'roc_score': 0.951048951048951, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9050802139037433, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.9202066590126292, 'f1': 0.9571428571428571, 'matthews': 0.26570779485078694}, 48: {'roc_score': 0.9169847328244274, 'f1': 0.9411764705882353, 'matthews': 0.5709722724341078}}, 'Locoregional control (Time)': {12: {'roc_score': 0.9505395683453237, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.9250954198473282, 'f1': 0.9458483754512635, 'matthews': 0.23680955876671803}, 36: {'roc_score': 0.9307086614173228, 'f1': 0.9407407407407408, 'matthews': 0.4214526991518474}, 48: {'roc_score': 0.9372370266479664, 'f1': 0.8545454545454544, 'matthews': 0.5122081309150257}}, 'FDM (months)': {12: {'roc_score': 0.8983451536643026, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.923076923076923, 'f1': 0.9537366548042704, 'matthews':

[tensor([5.4432e+03, 3.6144e+03, 8.2430e+01, 1.2737e+03, 5.9818e+03, 1.1084e+03,
         5.2678e+03, 7.2489e+03, 1.8122e+01, 1.4088e+03, 1.3374e+03, 7.5182e+03,
         5.3419e+03, 7.3724e+03, 1.8198e+03, 1.4183e+01, 1.1132e+03, 1.0193e+03,
         4.4800e+01, 3.6066e+02, 1.8748e+03, 2.5527e+03, 7.1148e+02, 7.9165e+03,
         2.8782e+03, 5.0530e+02, 3.0950e+00, 3.1622e+03, 1.4554e+03, 5.9490e+02,
         1.3486e+03, 3.6332e+02, 1.3261e+03, 5.5151e+00, 7.1273e+01, 4.2220e+00,
         2.0628e+02, 4.1137e+01, 4.8738e+02, 4.2708e+02, 3.4017e+01, 1.2636e+03,
         6.0903e+02, 1.9814e+03, 1.2396e+01, 3.0612e+00, 4.1793e+01, 6.1088e+00,
         2.1911e+01, 5.7798e+00, 2.2396e+01, 4.3527e+02, 6.1309e+00, 4.6436e+00,
         2.9885e+00, 4.5126e+00, 2.8672e+00, 3.7733e+00, 2.9371e+00, 3.4745e+00,
         3.0560e+00, 3.3860e+00, 3.7512e+01, 2.9493e+00, 2.9339e+00, 2.9181e+00,
         3.0339e+00, 2.8521e+00, 2.8475e+00, 8.0384e+01, 2.8904e+00, 3.0586e+02,
         3.0274e+00, 2.8198e

In [16]:
test2 = train_dsm(data,k=10,dist="LogNormal")
test2.time_to_event(xtr)

events torch.Size([4, 389])
events torch.Size([4, 147])
events torch.Size([4, 389])
best pretrain loss 19225.34473669207 epochs 259
val loss 1.3474606449799649
val metrics {'OS (Calculated)': {12: {'roc_score': 0.9108391608391608, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9685828877005347, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.9707233065442021, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.9475190839694656, 'f1': 0.9594095940959411, 'matthews': 0.5372569573006374}}, 'Locoregional control (Time)': {12: {'roc_score': 0.9100719424460432, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.9036259541984734, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.9173228346456692, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.9232117812061711, 'f1': 0.9561752988047808, 'matthews': 0.7029582191465814}}, 'FDM (months)': {12: {'roc_score': 0.946808510638298, 'f1': 0.9791666666666666, 'matthew

val loss 1.0064823352266015
val metrics {'OS (Calculated)': {12: {'roc_score': 0.9423076923076923, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9264705882352942, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.949483352468427, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.9561068702290076, 'f1': 0.9448818897637795, 'matthews': 0.6138824551033134}}, 'Locoregional control (Time)': {12: {'roc_score': 0.9514388489208634, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.9188931297709924, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.9393700787401575, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.929873772791024, 'f1': 0.8956521739130434, 'matthews': 0.5672290221003625}}, 'FDM (months)': {12: {'roc_score': 0.8900709219858156, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.9494833524684271, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.9474590869939707, 'f1

val loss 1.0571541215326665
val metrics {'OS (Calculated)': {12: {'roc_score': 0.951048951048951, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9431818181818181, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.9477611940298507, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.9303435114503817, 'f1': 0.9243027888446215, 'matthews': 0.5111355416958723}}, 'Locoregional control (Time)': {12: {'roc_score': 0.9631294964028777, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.9126908396946565, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.9251968503937008, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.8976157082748949, 'f1': 0.8947368421052632, 'matthews': 0.5874620626695477}}, 'FDM (months)': {12: {'roc_score': 0.8215130023640662, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.9110218140068886, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.9203273040482343, 'f

val loss 1.139151998805449
val metrics {'OS (Calculated)': {12: {'roc_score': 0.9562937062937062, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.9197860962566845, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.9276693455797933, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.9041030534351145, 'f1': 0.9322709163346613, 'matthews': 0.5675446442704167}}, 'Locoregional control (Time)': {12: {'roc_score': 0.9496402877697842, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.9188931297709924, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.9220472440944882, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.9126928471248247, 'f1': 0.9004329004329005, 'matthews': 0.5781688964173618}}, 'FDM (months)': {12: {'roc_score': 0.8853427895981087, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.9115958668197475, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.917312661498708, 'f1

[tensor([ 42.1063,  42.1573,  42.1145,  42.1386,  42.1601,  42.1800,  42.1137,
          42.0837,  42.1683,  42.1223,  42.0933,  42.0929,  42.0918,  42.0967,
          42.0891,  42.2180,  42.0924,  42.1403,  42.2761,  42.1377,  42.2181,
          42.1240,  42.5235,  42.0730,  42.1630,  42.1493,  42.2226,  42.0861,
          42.1175,  42.1457,  42.1005,  42.1339,  42.7625,  42.5807,  42.5661,
          43.8820,  42.1579,  43.0588,  42.5179,  42.7702,  42.1181,  42.2800,
          42.1307,  42.1374,  47.9287,  42.3837,  42.2205,  42.9575,  42.8907,
          47.3059,  42.6389,  42.3032,  42.0905,  43.0425,  45.0691,  45.5999,
          42.7033,  46.0689,  56.1420,  42.4532,  45.5622,  44.1205,  53.8388,
          61.7124,  50.1174,  44.4671,  48.3181,  59.6008,  60.1429,  55.1116,
          65.5586,  51.0782,  51.7256,  50.8291,  54.2523,  54.1132,  58.4665,
          65.7223,  56.4852,  61.2343,  49.2004,  53.7357,  93.1453,  56.9303,
          50.8168,  57.6640,  54.9969,  42.1801,  54